### Calculo de Ratios

#### liquidez
* Razón corriente = Activos corrientes / Pasivos corrientes
* Prueba ácida = (Activo corriente - Inventarios) / Pasivo corriente

#### Solvencia y endeudamiento
* Endeudamiento total = Total de pasivos / Total de activos
* Endeudamiento de largo plazo = Pasivo no corriente / Total de activos
* Cobertura de intereses = EBIT / Gastos por intereses

#### Rentabilidad
* ROA (Return on Assets) = Beneficio neto / Activo total
* ROE (Return on Equity) =  Beneficio neto / Patrimonio neto
* Margen de utilidad = 	Beneficio neto / Ventas (ingresos totales) (escrapeado "Margen del beneficio neto %")
* Rotación de activos = ingresos totales / Total de activos
* Rotación de inventarios = ingresos totales / Inventario

In [6]:
import sqlite3
import pandas as pd
pd.options.display.max_rows = 999

# Consultar datos
conn = sqlite3.connect('webmining.db')
cursor = conn.cursor()
cursor.execute("SELECT * FROM datos_anuales")
column_names = [description[0] for description in cursor.description]
filas = cursor.fetchall()

# Guardar los datos en un dataframe
df = pd.DataFrame(filas, columns=column_names)

In [7]:
indicadores = df.groupby(["informe","seccion","indicador"]).size()

print(indicadores)

informe     seccion            indicador                                                 
balance     activo_corriente   Crecimiento de efectivo y equivalentes                        74
                               Créditos totales                                              74
                               Efectivo restringido                                          29
                               Efectivo y equivalentes                                       74
                               Gastos pagados por adelantado                                 42
                               Inventario                                                    66
                               Inversiones a corto plazo                                     63
                               Otros activos corrientes                                      66
                               Otros créditos                                                41
                               Total de cuenta

In [8]:
# rubros que componen cada seccion
# no se incluye por ejemplo el rubro "otros creditos" ya que forma parte de "Creditos totales"
activo_corriente = [
    "Efectivo y equivalentes",
    "Inversiones a corto plazo",
    "Créditos totales",
    "Inventario",
    "Gastos pagados por adelantado",
    "Otros activos corrientes"
]
activo_no_corriente = [
    "Planta, propiedad y equipo, neto",
    "Inversiones a largo plazo",
    "Activos intangibles",
    "Total de otros activos"
]
pasivo_corriente = [
    "Total de cuentas por pagar",
    "Total de gastos devengados",
    "Préstamos a corto plazo",
    "Parte actual de la deuda a largo plazo/arrendamientos",
    "Total de otros pasivos corrientes"
]
pasivo_no_corriente = [
    "Deuda a largo plazo",
    "Arrendamientos a largo plazo",
    "Total de otros pasivos"
]
patrimonio_neto = [
    "Total de acciones preferentes",
    "Acciones ordinarias y APIC",
    "Ganancias retenidas",
    "Acciones propias y otras",
    "Intereses minoritarios"
]

inventario = ["Inventario"]

EBIT = ["EBIT"]
	
intereses = ["Total de gastos de intereses"]

In [9]:
# features de indicadores ya escrapeados

indicadores_scrapeados = [
  "Crecimiento de efectivo y equivalentes",
  "Crecimiento de activos totales",
  "Crecimiento del pasivo total",
  "Crecimiento de la deuda total",
  "Crecimiento del beneficio neto",
  "Crecimiento del EBITDA",
  "Crecimiento de los ingresos totales"
]

In [10]:
ac = pd.pivot_table(df[df.indicador.isin(activo_corriente)], index=["company","ejercicio"], values="valor").rename(columns={"valor":"activo_corriente"})
anc = pd.pivot_table(df[df.indicador.isin(activo_no_corriente)], index=["company","ejercicio"], values="valor").rename(columns={"valor":"activo_no_corriente"})
pc = pd.pivot_table(df[df.indicador.isin(pasivo_corriente)], index=["company","ejercicio"], values="valor").rename(columns={"valor":"pasivo_corriente"})
pnc = pd.pivot_table(df[df.indicador.isin(pasivo_no_corriente)], index=["company","ejercicio"], values="valor").rename(columns={"valor":"pasivo_no_corriente"})
pn = pd.pivot_table(df[df.indicador.isin(patrimonio_neto)], index=["company","ejercicio"], values="valor").rename(columns={"valor":"patrimonio_neto"})
inv = pd.pivot_table(df[df.indicador.isin(inventario)], index=["company","ejercicio"], values="valor").rename(columns={"valor":"inventario"})
ebit = pd.pivot_table(df[df.indicador.isin(EBIT)], index=["company","ejercicio"], values="valor").rename(columns={"valor":"EBIT"})
int = pd.pivot_table(df[df.indicador.isin(intereses)], index=["company","ejercicio"], values="valor").rename(columns={"valor":"intereses"})

data = pd.concat([ac,anc,pc,pnc,pn,inv,ebit,int], axis=1)
data

activo_corriente  activo_no_corriente  \
company                ejercicio                                          
3m-co                  2020             214.283833            61.537000   
                       2021             183.053167            73.351750   
                       2022             173.277000           249.450000   
                       2023              68.330833            60.997750   
                       2024             200.116333             5.996000   
amazon-com-inc         2020              73.100000            47.115500   
                       2021              80.667600            64.742250   
                       2022              84.072167            78.971000   
                       2023             111.475667            88.875750   
                       2024              72.936667           108.506750   
apple-computer-inc     2020              28.742600            60.058333   
                       2021              26.967200            72.055333   
                       2022              27.081000            72.450000   
                       2023              28.713200            69.672333   
                       2024              30.597400            70.664333   
berkshire-hathaway-inc 2020             173.870000           169.925750   
                       2021             211.647600           187.673750   
                       2022              40.682600           186.263000   
                       2023             239.951400           206.066500   
                       2024             250.919800           186.608500   
chevron                2020              82.434833            53.428000   
                       2021             108.686500            51.449250   
                       2022             179.552500            51.841500   
                       2023              85.276167            55.126000   
                       2024              74.584000            54.006750   
coca-cola-co           2020              28.514667            17.014000   
                       2021               3.757333            17.952250   
                       2022             138.297167            17.543000   
                       2023               4.455333            17.742750   
                       2024             104.732333            18.638000   
disney                 2020             227.344000            41.574500   
                       2021             212.460000            42.488000   
                       2022               7.274500            43.633250   
                       2023               8.190750            43.204000   
                       2024               6.310250            42.744500   
google-inc             2020             180.313600            36.330000   
                       2021              37.628600            42.781250   
                       2022              41.198750            50.117250   
                       2023              42.882500            57.715500   
                       2024              40.927750            71.636250   
johnson-johnson        2021              10.247400            30.914250   
                       2022              12.195800            30.259750   
                       2023               9.892800            33.021000   
                       2024              94.495200            31.052750   
microsoft-corp         2020              36.383000            29.849000   
                       2021              36.881200            37.343250   
                       2022              33.936800            48.789000   
                       2023              36.851400            56.929750   
                       2024              31.946800            88.107250   
nike                   2020             143.952000           169.096333   
                       2021              61.824333           173.979333   
                       2022             149.224167         

In [9]:
df[
(df["informe"] == "balance") &
(df["seccion"] == "patrimonio_neto") &
(df["ejercicio"] == 2020) &
(df["company"] == "apple-computer-inc")
].to_excel("apple_2020.xlsx", index=False)


In [3]:
df["company"].unique()

array(['apple-computer-inc', 'microsoft-corp', 'google-inc',
       'tesla-motors', 'visa-inc', 'berkshire-hathaway-inc',
       'johnson-johnson', 'pfizer', 'amazon-com-inc', 'disney', 'nike',
       'procter-gamble', 'chevron', '3m-co', 'coca-cola-co'], dtype=object)

In [7]:
df["seccion"].unique()

array(['activo_corriente', 'activo_total', 'pasivo_corriente',
       'pasivo_total', 'patrimonio_neto', 'crecimiento_deuda', ''],
      dtype=object)